In [2]:
!pip install feature-engine

     |████████████████████████████████| 179 kB 5.2 MB/s 
     |████████████████████████████████| 9.5 MB 66.3 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import joblib
import sys
import itertools
import numpy as np 
import pandas as pd
import re
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from feature_engine.encoding import OneHotEncoder as fe_OneHotEncoder
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

In [69]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lambda DS/04.Unit_4/04.Build_Week_Spotify/tracks_features.csv')

In [70]:
def explicit_converter(x):
    if x:
        return 1 
    else:
        return 0

In [71]:
def decade_function(x):
    if x in range(1900,1991):
        return '1900-1990'
    elif x in range(1991,1996):
        return '1991-1995'
    elif x in range(1996,2001):
        return '1996-2000'
    elif x in range(2001,2006):
        return '2001-2005'
    elif x == 2006:
        return '2006' 
    elif x == 2007:
        return '2007'
    elif x == 2008:
        return '2008'
    elif x == 2009:
        return '2009'
    elif x == 2010:
        return '2010'
    elif x == 2011:
        return '2011'
    elif x == 2012:
        return '2012'
    elif x == 2013:
        return '2013'
    elif x == 2014:
        return '2014'
    elif x == 2015:
        return '2015'
    elif x == 2016:
        return '2016'
    elif x == 2017:
        return '2017'
    elif x == 2018:
        return '2018'
    elif x == 2019:
        return '2019'
    elif x == 2020:
        return '2020'
    elif x == 2021:
        return '2021'
        

In [72]:
def wrangle(df3):
    df3['artists_new'] = df3['artists'].apply(lambda x: re.findall(r"'([^']*)'", x))
    df3['artists_new2'] = df3['artists'].apply(lambda x: re.findall('\"(.*?)\"',x))
    df3['artists_final'] = np.where(df3['artists_new'].apply(lambda x: not x), df3['artists_new2'], df3['artists_new'])
    df3['artists_song'] = df3.apply(lambda row: row['artists_final'][0]+row['name'],axis = 1)
    df3.drop_duplicates('artists_song',inplace = True)
    cols_to_drop = ['id', 'album_id','artists', 'artist_ids', 'track_number', 'disc_number', 'release_date', 'artists_new', 'artists_new2', 'artists_song', 'duration_ms']
    df3.drop(columns = cols_to_drop, inplace = True)
    #df3['Genres'] = df3['Genres'].apply(lambda x: [re.sub(' ','_',i) for i in re.findall(r"'([^']*)'", str(x))]) 
    df3['explicit'] = df3['explicit'].apply(explicit_converter)
    #df3.rename(columns={"Genres": "genres"}, inplace = True)
    df3['year'] = df3['year'].replace(0, np.nan)
    df3.dropna(subset = ['year'], inplace = True)
    return df3

In [73]:
wrangle(df)

,name,album,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_final
0,Testify,The Battle Of Los Angeles,0,0.470,0.978,7,-5.399,1,0.0727,0.02610,0.000011,0.3560,0.503,117.906,4.0,1999.0,[Rage Against The Machine]
1,Guerrilla Radio,The Battle Of Los Angeles,1,0.599,0.957,11,-5.764,1,0.1880,0.01290,0.000071,0.1550,0.489,103.680,4.0,1999.0,[Rage Against The Machine]
2,Calm Like a Bomb,The Battle Of Los Angeles,0,0.315,0.970,7,-5.424,1,0.4830,0.02340,0.000002,0.1220,0.370,149.749,4.0,1999.0,[Rage Against The Machine]
3,Mic Check,The Battle Of Los Angeles,1,0.440,0.967,11,-5.830,0,0.2370,0.16300,0.000004,0.1210,0.574,96.752,4.0,1999.0,[Rage Against The Machine]
4,Sleep Now In the Fire,The Battle Of Los Angeles,0,0.426,0.929,2,-6.729,1,0.0701,0.00162,0.105000,0.0789,0.539,127.059,4.0,1999.0,[Rage Against The Machine]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1204020,Gospel of Juke,Notch - EP,0,0.264,0.966,5,-6.970,0,0.0672,0.00935,0.002240,0.3370,0.415,159.586,4.0,2014.0,[FVLCRVM]
1204021,Prism Visions,Notch - EP,0,0.796,0.701,11,-6.602,0,0.0883,0.10400,0.644000,0.0749,0.781,121.980,4.0,2014.0,[FVLCRVM]
1204022,Tokyo 360,Notch - EP,0,0.785,0.796,9,-5.960,0,0.0564,0.03040,0.918000,0.0664,0.467,121.996,4.0,2014.0,[FVLCRVM]
1204023,Yummy!,Notch - EP,0,0.665,0.856,6,-6.788,0,0.0409,0.00007,0.776000,0.1170,0.227,124.986,4.0,2014.0,[FVLCRVM]


In [10]:
df['age'] = df['year'].apply(np.int64)

In [11]:
df['age'] = df['age'].apply(decade_function)

In [12]:
ohe_enc = fe_OneHotEncoder(
    top_categories=None,
    variables=['age'], drop_last=True)  # to return k-1, false to return k


ohe_enc.fit(df.fillna('Missing'))

OneHotEncoder(drop_last=True, drop_last_binary=False, ignore_format=False,
              top_categories=None, variables=['age'])

In [13]:
df = ohe_enc.transform(df.fillna('Missing'))

In [74]:
df.shape

(1126176, 17)

In [75]:
#feature scailing for float columns in df
scale_cols = ['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'key', 'mode']
scaled = df[scale_cols].reset_index(drop=True)
scaler = MinMaxScaler()
scaled_float_df = pd.DataFrame(scaler.fit_transform(scaled), columns = scaled.columns)

In [76]:
dropper = ['danceability', 'energy', 'loudness', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'time_signature', 'year', 'key', 'mode']

In [77]:
df1 = df.drop(columns = dropper)

In [78]:
df = df.reset_index()
df1 = df1.reset_index()
scaled_float_df.reset_index()

,index,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,key,mode
0,0,0.470,0.978,0.812104,0.075026,0.026205,0.000011,0.3560,0.503,0.473644,0.8,0.636364,1.0
1,1,0.599,0.957,0.806675,0.194014,0.012952,0.000071,0.1550,0.489,0.416496,0.8,1.000000,1.0
2,2,0.315,0.970,0.811732,0.498452,0.023494,0.000002,0.1220,0.370,0.601561,0.8,0.636364,1.0
3,3,0.440,0.967,0.805694,0.244582,0.163655,0.000004,0.1210,0.574,0.388665,0.8,1.000000,0.0
4,4,0.426,0.929,0.792322,0.072343,0.001627,0.105000,0.0789,0.539,0.510412,0.8,0.181818,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126171,1126171,0.264,0.966,0.788738,0.069350,0.009388,0.002240,0.3370,0.415,0.641078,0.8,0.454545,0.0
1126172,1126172,0.796,0.701,0.794211,0.091125,0.104418,0.644000,0.0749,0.781,0.490009,0.8,1.000000,0.0
1126173,1126173,0.785,0.796,0.803760,0.058204,0.030522,0.918000,0.0664,0.467,0.490074,0.8,0.818182,0.0
1126174,1126174,0.665,0.856,0.791445,0.042208,0.000070,0.776000,0.1170,0.227,0.502085,0.8,0.545455,0.0


In [79]:
df2 = pd.concat([df1, scaled_float_df], axis = 1)

In [91]:
# Drop unnecessary categories
df_train = df2.drop(['name', 'album', 'artists_final', 'index'], axis = 1)

In [101]:
df.drop(columns='index', inplace=True)

# Nearest Neighbors (Brute Force Algo)

In [92]:
# Fit on DTM
nn = NearestNeighbors(n_neighbors=6, algorithm= 'brute', n_jobs = -1)
nn.fit(df_train)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=6, p=2, radius=1.0)

In [93]:
# sample a doc from dtm to use as our query point 
doc_index = 1126175
doc = [df_train.iloc[doc_index].values]

# Query Using kneighbors 
neigh_dist, neigh_index = nn.kneighbors(doc)

print(neigh_dist)
print(neigh_index)

[[0.         0.15715287 0.1667135  0.17779515 0.2002936  0.21819788]]
[[1126175  897492  231125  226170  911316  807530]]


In [102]:
df.iloc[897492]

name                   Different You - Approaching Black Extended Mix
album               Different You (Approaching Black, Movement Mac...
explicit                                                            0
danceability                                                    0.712
energy                                                          0.636
key                                                                 3
loudness                                                       -8.803
mode                                                                0
speechiness                                                     0.037
acousticness                                                   0.0547
instrumentalness                                                0.362
liveness                                                        0.306
valence                                                         0.265
tempo                                                         119.987
time_signature      

In [58]:
df.iloc[259048]

name                                            Maria
album               The Best Of - Ultimate Collection
explicit                                            0
danceability                                    0.776
energy                                           0.78
key                                                10
loudness                                        -6.95
mode                                                0
speechiness                                    0.0825
acousticness                                 0.000216
instrumentalness                                    0
liveness                                         0.25
valence                                         0.732
tempo                                         112.523
time_signature                                      4
year                                             2004
artists_final                                [Vico C]
age_1996-2000                                       0
age_1991-1995               

# Pickle the Model

In [96]:
model_filename = 'knn_model' 
joblib.dump(nn, open(model_filename, 'wb'))